In [ ]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [ ]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [ ]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test

In [ ]:
pivotsearch = :full
seed = 1234

Random.seed!(seed)

R = 15
abstol = 1e-4
grid = QD.DiscretizedGrid{1}(R, (0.0,), (log(20.),))

rindex = [rand(1:2, R) for _ in 1:100]

f(bitlist) = any(bitlist .> 2) ? 0.0 : fx(QD.quantics_to_origcoord(grid, bitlist)[1])
rpoint = Float64[QD.quantics_to_origcoord(grid, r)[1] for r in rindex]

B = 2^(-30) # global variable

function fx(x)
    #return cos(x / B) * cos(x / (4 * sqrt(5) * B)) * exp(-x^2) + 2 * exp(-x)
    cos(x^2)
end

localdims = fill(2, R)
firstpivot = ones(Int, R)
tci, ranks, errors = crossinterpolate2(
    Float64,
    f,
    localdims,
    [firstpivot];
    tolerance=abstol,
    maxbonddim=1000,
    maxiter=20,
    loginterval=1,
    verbosity=1,
    normalizeerror=false,
    pivotsearch=pivotsearch,
)
@show abstol

In [ ]:
for b in 1:length(tci)-1
    @show length(TCI.Iset(tci, b+1)), length(TCI.Jset(tci, b))
end

In [ ]:
for orthocenter in [1, length(tci), length(tci) ÷ 2]
    tt = TCI.TensorTrain(TCI.sitetensors(tci, f; orthocenter=orthocenter))
    println("Orthocenter: ", orthocenter)
    for b in 1:length(tci)-1
        Iset = TCI.Iset(tci, b+1)
        Jset = TCI.Jset(tci, b)
        println(b, "    ", maximum(abs, [TCI.evaluate(tt, vcat(i, j)) - f(vcat(i, j)) for i in Iset, j in Jset]))
    end
    println("error on global pivots: ", maximum([abs(TCI.evaluate(tt, p) - f(p)) for p in tci.globalpivots]))
end

In [ ]:
orthocenter = 1
tt = TCI.TensorTrain(TCI.sitetensors(tci, f; orthocenter=orthocenter))
println("Orthocenter: ", orthocenter)
b = 1
Iset = TCI.Iset(tci, b+1)
Jset = TCI.Jset(tci, b)
@show length(Iset), length(Jset)
for (idxi, i) in enumerate(Iset), (idxj, j) in enumerate(Jset)
    diff =  abs(TCI.evaluate(tt, vcat(i, j)) - f(vcat(i, j)))
    if diff > 1e-9
        println(idxi, " ", idxj, " ", i, " ", j, " ", abs(TCI.evaluate(tt, vcat(i, j)) - f(vcat(i, j))), " ", vcat(i, j) ∈ tci.globalpivots)
    end
end

In [ ]:
tt = TCI.TensorTrain(TCI.sitetensors(tci, f))
tt_old = TCI.TensorTrain(TCI.sitetensors_site0update(tci, f))

@show TCI.linkdims(tt)
@show TCI.linkdims(tt_old)

In [ ]:
for b in 1:length(tci)-1
    Iset = TCI.Iset(tci, b+1)
    Jset = TCI.Jset(tci, b)
    println(b, "    ", maximum(abs, [TCI.evaluate(tt, vcat(i, j)) - f(vcat(i, j)) for i in Iset, j in Jset]))
end

In [ ]:
for l in 1:length(tci)
    Iset = TCI.Iset(tci, l)
    Jset = TCI.Jset(tci, l)
    println(l, "    ", maximum(abs, [TCI.evaluate(tt, vcat(i, l, j)) - f(vcat(i, l, j)) for i in Iset, j in Jset, l in 1:localdims[l]]))
end

In [ ]:
maximum([abs(TCI.evaluate(tt, p) - f(p)) for p in tci.globalpivots])

In [ ]:
for b in 1:length(tci)-1
    Iset = TCI.Iset(tci, b+1)
    Jset = TCI.Jset(tci, b)
    println(b, "    ", maximum(abs, [TCI.evaluate(tt_old, vcat(i, j)) - f(vcat(i, j)) for i in Iset, j in Jset]))
end

maximum([abs(TCI.evaluate(tt_old, p) - f(p)) for p in tci.globalpivots])

In [ ]:
using Plots

p = plot(ylims=(0, 2))
b = 5
plot!(p, sort(vec(abs.(tt[b]))))
plot!(p, sort(vec(abs.(tt_old[b]))))

In [ ]:
length(tci.globalpivots)